In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

# Preparación de datos

* descomprimmos los zip y los separamos en entrenamiento y validación

In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

* usamos los nombres de las carpetas como etiquetas para un clasificador binario (el caso que nos atañe)

In [0]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/validation-horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/validation-humans')

# Contruimos el modelo

In [0]:
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

* Vamos a ver la red en detalle


In [0]:
model.summary()

En este caso utilizamos el optimizador
[RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) mejor que SGD () [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) ,ya que una de las propiedas con Adam o Adagrad es qeu actualizan el learning rate de forma automatica para encontrar el minimó optimo.

[Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam)
[Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad)

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

# Data Augmentation (dame más!!!!)

En este ejercicio vamos a usar generadores para entrenar la red, de manera que en momoria tendremos diferentes versiones de una misma imagén a la que hemos aplicado un conjunto de transformaciones.

Todo esto lo hacemos de una forma sencilla gracias a ImageDataGenerator de Keras (que no os mientan esto no lo tenía TF).

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Escalamos/normalizamos las imagenes. En algunos casos vereis que le restan 0.5 a mayores, suele ser por el uso de otro tipo de función de activación.
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Crea imágenes en grupos de 128
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  
        target_size=(150, 150),  
        batch_size=128,
        class_mode='binary') # OJO con esto

# Idem para el set de validación. La colección es más pequeña, merece la pena reducir el tamaño del batch.
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  
        target_size=(150, 150),  
        batch_size=32,        
        class_mode='binary')

# Entrenamos

Aquí es donde tenemos una de las grandes diferencias entre el modelo que vimos fashion MNist e imagenes reales que requiren pre-procesado. 

En este caso usamos fit_generator. 

Este metodo de la API de TF+Keras trabaja direcatemente sobre las imagenes de entrada, las procesa en memoria y las lanza a la red como una más. En el ImageDataGen podemos incluir que nos las guarde en disco para no tener que volver a procesarlas.



In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Busca un caballo (o una persona)


In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

# Mirando por dentro

Vamos a ver de un conjunto de imagenes cual es su forma o su render en las diferentes capas de la red.

In [0]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])



successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)


img = load_img(img_path, target_size=(150, 150))  # load_img viene de PIL otra libería de python para imágenes
x = img_to_array(img)  
x = x.reshape((1,) + x.shape)  
x /= 255

# Extremos el mapa y el nombre de las capas
successive_feature_maps = visualization_model.predict(x)
layer_names = [layer.name for layer in model.layers]

# Render
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    n_features = feature_map.shape[-1]  
    size = feature_map.shape[1]
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='inferno')

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)